In [1]:
import psycopg2

In [2]:
try:
    conn = psycopg2.connect("dbname=postgres user=postgres host=localhost password=***** port=5432")
except psycopg2.Error as e:
    print("error with connection")
    print(e)
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("error with setting cursor")
    print(e)
conn.set_session(autocommit=True)

we use denormalization for low latency and lower response time <br> also when analysing or using the data

In [3]:
create = "CREATE TABLE IF NOT EXISTS "
try:
    cur.execute(create + "album_liberary(album_id int, album_name varchar, artist_id int, year int)")
except psycopg2.Error as e:
    print("error with CREATE table")
    print(e)
try:
    cur.execute(create + "song_liberary(song_id int, song_name varchar, album_id int)")
except psycopg2.Error as e:
    print("error with CREATE table")
    print(e)
try:
    cur.execute(create + "artist_liberary(artist_id int, artist_name varchar)")
except psycopg2.Error as e:
    print("error with CREATE table")
    print(e)
try:
    cur.execute(create + "song_length(song_id int, length int)")
except psycopg2.Error as e:
    print("error with CREATE table")
    print(e)

In [7]:
inset_into_album = "INSERT INTO album_liberary (album_id, album_name, artist_id, year)\
    VALUES(%s,%s,%s,%s)"
inset_into_song = "INSERT INTO song_liberary (song_id, song_name, album_id)\
    VALUES(%s,%s,%s)"
inset_into_artist = "INSERT INTO artist_liberary (artist_id, artist_name)\
    VALUES(%s,%s)"
inset_into_lenght = "INSERT INTO song_length (song_id, length)\
    VALUES(%s,%s)"

In [9]:
album_values = [(1, "Rubber Soul", 1, 1965), (2, "let it be", 1, 1970)]
song_values = [(1, "Mechele", 1), (2, "think", 1), (3, "in my life", 1), (4, "let it be", 2), (5, "across", 2)]
artist_values = [(1, "the beatles")]
lenght_values = [(1, 123), (2, 343), (3, 212), (4, 221), (5, 121)]

In [10]:
for v in album_values:
    try:
        cur.execute(inset_into_album, v)
    except psycopg2.Error as e:
        print(e)
for v in song_values:
    try:
        cur.execute(inset_into_song, v)
    except psycopg2.Error as e:
        print(e)
for v in artist_values:
    try:
        cur.execute(inset_into_artist, v)
    except psycopg2.Error as e:
        print(e)
for v in lenght_values:
    try:
        cur.execute(inset_into_lenght, v)
    except psycopg2.Error as e:
        print(e)


In [11]:
try:
    cur.execute("SELECT * FROM album_liberary")
except psycopg2.Error as e:
    print(e)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
print("_" * 18)
try:
    cur.execute("SELECT * FROM song_liberary")
except psycopg2.Error as e:
    print(e)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
print("_" * 18)
try:
    cur.execute("SELECT * FROM artist_liberary")
except psycopg2.Error as e:
    print(e)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
print("_" * 18)
try:
    cur.execute("SELECT * FROM song_length")
except psycopg2.Error as e:
    print(e)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
print("_" * 18)


(1, 'Rubber Soul', 1, 1965)
(2, 'let it be', 1, 1970)
__________________
(1, 'Mechele', 1)
(2, 'think', 1)
(3, 'in my life', 1)
(4, 'let it be', 2)
(5, 'across', 2)
__________________
(1, 'the beatles')
__________________
(1, 123)
(2, 343)
(3, 212)
(4, 221)
(5, 121)
__________________


In [21]:
denormalized = cur.execute("SELECT artist_liberary.artist_id, artist_name, album_liberary.album_id, \
        album_name, year, song_liberary.song_id, song_name, song_length.length \
    FROM (((artist_liberary JOIN album_liberary ON artist_liberary.artist_id=album_liberary.artist_id)\
            JOIN song_liberary ON album_liberary.album_id=song_liberary.album_id)\
                JOIN song_length ON song_liberary.song_id=song_length.song_id)")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
print("_" * 18)

(1, 'the beatles', 1, 'Rubber Soul', 1965, 1, 'Mechele', 123)
(1, 'the beatles', 1, 'Rubber Soul', 1965, 2, 'think', 343)
(1, 'the beatles', 1, 'Rubber Soul', 1965, 3, 'in my life', 212)
(1, 'the beatles', 2, 'let it be', 1970, 4, 'let it be', 221)
(1, 'the beatles', 2, 'let it be', 1970, 5, 'across', 121)
__________________


with denormalization we think about the queries to reduce the number of joins in reading to lessen the response time